In [15]:
import os
from dagster import asset, AssetIn, Output, WeeklyPartitionsDefinition
import pandas as pd
from pyspark.sql.dataframe import DataFrame
from pyspark.sql import SparkSession
from pyspark.sql.functions import monotonically_increasing_id, lit, concat

In [3]:
spark = SparkSession.builder.master("local[*]").getOrCreate()

23/12/14 16:17:34 WARN Utils: Your hostname, TrungUbun resolves to a loopback address: 127.0.1.1; using 192.168.1.6 instead (on interface enp3s0)
23/12/14 16:17:34 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/14 16:17:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/12/14 16:17:40 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [11]:
spark_df = spark.read.csv("green_tripdata_2023-01.csv", inferSchema = True, header = True)
spark_df.show()

+---+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|_c0|VendorID|lpep_pickup_datetime|lpep_dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+---+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|  0|       2| 2023-01-01 00:26:10|  2023-01-01 00:37:11|                 N|       1.0|         166|         143|            1.0|         2.58|       1

23/12/14 16:27:13 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , VendorID, lpep_pickup_datetime, lpep_dropoff_datetime, store_and_fwd_flag, RatecodeID, PULocationID, DOLocationID, passenger_count, trip_distance, fare_amount, extra, mta_tax, tip_amount, tolls_amount, ehail_fee, improvement_surcharge, total_amount, payment_type, trip_type, congestion_surcharge
 Schema: _c0, VendorID, lpep_pickup_datetime, lpep_dropoff_datetime, store_and_fwd_flag, RatecodeID, PULocationID, DOLocationID, passenger_count, trip_distance, fare_amount, extra, mta_tax, tip_amount, tolls_amount, ehail_fee, improvement_surcharge, total_amount, payment_type, trip_type, congestion_surcharge
Expected: _c0 but found: 
CSV file: file:///home/trung/Study/KHDL/NYC-TripRecord/Test/green_tripdata_2023-01.csv


In [16]:
spark_df.cache()
select_cols = ["lpep_dropoff_datetime", "DOLocationID"]
spark_df = spark_df.select(select_cols)
spark_df = spark_df.dropDuplicates(select_cols)
specialID = concat(lit("F2023"), monotonically_increasing_id())
spark_df = spark_df.withColumn("PickUpID", specialID)
spark_df = spark_df.withColumnRenamed('tpep_dropoff_datetime','Dropoff_datetime')
spark_df.show()
spark_df.unpersist()

+---------------------+------------+--------+
|lpep_dropoff_datetime|DOLocationID|PickUpID|
+---------------------+------------+--------+
|  2023-01-01 00:39:24|         146|  F20230|
|  2023-01-01 01:41:21|         140|  F20231|
|  2023-01-01 03:06:10|          48|  F20232|
|  2023-01-01 02:03:43|           7|  F20233|
|  2023-01-01 07:23:19|         142|  F20234|
|  2023-01-01 08:48:49|         223|  F20235|
|  2023-01-01 11:40:34|          65|  F20236|
|  2023-01-01 15:45:23|          74|  F20237|
|  2023-01-01 21:24:19|          41|  F20238|
|  2023-01-01 23:04:39|         243|  F20239|
|  2023-01-02 12:25:39|         186| F202310|
|  2023-01-02 12:05:34|          95| F202311|
|  2023-01-02 12:21:16|         244| F202312|
|  2023-01-02 14:11:25|         223| F202313|
|  2023-01-02 13:34:23|         179| F202314|
|  2023-01-02 14:24:04|         168| F202315|
|  2023-01-02 16:07:21|         166| F202316|
|  2023-01-02 16:16:37|         262| F202317|
|  2023-01-03 07:27:47|          4

DataFrame[lpep_dropoff_datetime: timestamp, DOLocationID: int, PickUpID: string]